## Лабораторная работа 2

In [442]:
import pandas as pd
import numpy as np
import itertools
import math
from IPython.display import display
from cmath import isnan, nan
from numpy import dot
from numpy.linalg import norm

#### Получить исходные данные

In [ ]:
pd.set_option('display.max_columns', None)
df0 = pd.read_csv('../lab_01/perfume/data.csv', delimiter=',', encoding="utf8")
df = df0.copy(deep=True)

#### Получить характерные для семейств запахи

In [443]:
smellLike = pd.read_csv('../lab_01/perfume/smells.csv', delimiter=',', encoding="utf8")

smellLike["семейство"] = smellLike["семейство"].map(lambda elem: str(elem).lower())
smellLike["ассоциации"] = smellLike["ассоциации"].map(lambda elem: str(elem).lower().split(';')) + \
                          smellLike["характеристики"].map(lambda elem: str(elem).lower().split(';'))

itemAss = list(set(itertools.chain.from_iterable(smellLike["ассоциации"].values)))
itemAss.remove('nan')
itemAss.sort()

for item in itemAss:
    smellLike[item] = smellLike["ассоциации"].map(lambda elem: 1 if item in elem else 0)
del smellLike["ассоциации"]
del smellLike["характеристики"]
smellLike


,семейство,активный,амбра,апельсин,белый чай,бергамот,благовоние,бодрящий,ваниль,виноград,гвоздика,герань,горный воздух,грейпфрут,древесный,дым,жизнерадостный,зеленое яблоко,зеленый чай,земля,игривый,имбирь,интенсивный,карамель,кедр,корица,кофе,куркума,лаванда,ладан,лайм,легкий,лимон,листья,массивный,металл,морозная свежесть,морской,мох,мускус,мягкий,насыщенный,ненавязчивый,необычный,огурец,орех,острый,папоротник,перец,персик,плотный,притягательный,пряно-горький,пряный,роза,роскошный,сандал,свежий,сладкий,сливочный,смола,согревающий,спокойный,спортивный,сухая трава,сухой,табак,таинственный,телесный,теплый,терпкий,томный,тонизирующий,тонкий,торжественный,трава,тяжелый,черный чай,чувственный,шоколад,энергичный,яркий
0,восточные,0,1,0,0,0,1,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,1,1,0,0
1,древесные,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,1,1,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0
2,цитрусовые,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1
3,фруктовые,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,свежие,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
5,шипровые,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0
6,пряные,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,цветочные,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,фужерные,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
9,мускусные,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0


#### Преобразовать параметры (пол, шлейф, концентрация, сегмент)

In [444]:
sexDict = { "м": 0, "у": 0.5, "ж": 1 }
df["пол"] = df["пол"].map(lambda elem: sexDict[elem])

In [445]:
tailDict = { "крайне незаметный": 0, "незаметный": 0.25, "средний": 0.5, "заметный": 0.75, "очень заметный": 1}
df["шлейф"] = df["шлейф"].map(lambda elem: tailDict[elem])

In [446]:
conDict = {"освежающая вода": 0, "одеколон": 0.2, "туалетная вода": 0.4, "парфюмерная вода": 0.6, "духи": 0.8, "масляные духи": 1}
df["концентрация"] = df["концентрация"].map(lambda elem: conDict[elem])

In [447]:
segmentDict = {"масс-маркет": 0, "люкс": 0.5, "нишевая парфюмерия": 1}
df["сегмент"] = df["сегмент"].map(lambda elem: segmentDict[elem])

#### Задать матрицу смежности для парфюмерных школ

In [448]:
countryDict = {"Франция": 0, "Италия": 1, "ОАЭ": 2, "США": 3, "Россия": 4, "Германия": 5}

countryMatr = np.zeros((len(countryDict), len(countryDict)))
countryMatr[countryDict["Франция"]][countryDict["Италия"]] = countryMatr[countryDict["Италия"]][countryDict["Франция"]] = 0.5
countryMatr[countryDict["Франция"]][countryDict["ОАЭ"]] = countryMatr[countryDict["ОАЭ"]][countryDict["Франция"]] = 0.9
countryMatr[countryDict["Франция"]][countryDict["США"]] = countryMatr[countryDict["США"]][countryDict["Франция"]] = 0.9
countryMatr[countryDict["Франция"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["Франция"]] = 0.8
countryMatr[countryDict["Франция"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["Франция"]] = 1

countryMatr[countryDict["Италия"]][countryDict["ОАЭ"]] = countryMatr[countryDict["ОАЭ"]][countryDict["Италия"]] = 0.8
countryMatr[countryDict["Италия"]][countryDict["США"]] = countryMatr[countryDict["США"]][countryDict["Италия"]] = 0.5
countryMatr[countryDict["Италия"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["Италия"]] = 0.9
countryMatr[countryDict["Италия"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["Италия"]] = 1

countryMatr[countryDict["ОАЭ"]][countryDict["США"]] = countryMatr[countryDict["США"]][countryDict["ОАЭ"]] = 0.9
countryMatr[countryDict["ОАЭ"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["ОАЭ"]] = 0.8
countryMatr[countryDict["ОАЭ"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["ОАЭ"]] = 1

countryMatr[countryDict["США"]][countryDict["Россия"]] = countryMatr[countryDict["Россия"]][countryDict["США"]] = 0.7
countryMatr[countryDict["США"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["США"]] = 1

countryMatr[countryDict["Россия"]][countryDict["Германия"]] = countryMatr[countryDict["Германия"]][countryDict["Россия"]] = 1

#### Преобразование оставшихся данных

In [449]:
df["верхние ноты"] = df["верхние ноты"].map(lambda elem: str(elem).lower().split(';'))
df["ноты сердца"] = df["ноты сердца"].map(lambda elem: str(elem).lower().split(';'))
df["базовые ноты"] = df["базовые ноты"].map(lambda elem: str(elem).lower().split(';'))
df["запахи"] = df["запахи"].map(lambda elem: str(elem).lower().split(';'))
df["семейства"] = df["семейства"].map(lambda elem: str(elem).lower().split(';'))
df["время года"] = df["время года"].map(lambda elem: str(elem).lower().split(';'))
df["характеристики аромата"] = df["характеристики аромата"].map(lambda elem: str(elem).lower().split(';'))

In [450]:
dfTree = pd.DataFrame(columns=["название", "семейства"], data=df[["название", "семейства"]].values)

In [451]:
df["возраст (до 18)"] = df["возраст (до 18)"].map(lambda elem: 1 if elem == '+' else 0)
df["возраст (до 26)"] = df["возраст (до 26)"].map(lambda elem: 1 if elem == '+' else 0)
df["возраст (до 35)"] = df["возраст (до 35)"].map(lambda elem: 1 if elem == '+' else 0)
df["возраст (до 45)"] = df["возраст (до 45)"].map(lambda elem: 1 if elem == '+' else 0)
df["возраст (после 45)"] = df["возраст (после 45)"].map(lambda elem: 1 if elem == '+' else 0)

In [452]:
df["стойкость минимум (ч)"] = df["стойкость минимум (ч)"].values / max(df["стойкость минимум (ч)"].values)
df["стойкость максимум (ч)"] = df["стойкость максимум (ч)"].values / max(df["стойкость максимум (ч)"].values)

In [453]:
df["цена за 1 мл"] = df["цена"] / df["мл"]
df["цена за 1 мл"] = (df["цена за 1 мл"].values - min(df["цена за 1 мл"].values)) / (max(df["цена за 1 мл"].values) - min(df["цена за 1 мл"].values))

In [454]:
itemSeason = list(set(itertools.chain.from_iterable(df["время года"].values)))
itemSeason.remove('nan')
for item in itemSeason:
    df[item] = df["время года"].map(lambda elem: 1 if item in elem else 0)

In [455]:
itemSmells = list(set(itertools.chain.from_iterable(df["верхние ноты"].values)) | 
             set(itertools.chain.from_iterable(df["ноты сердца"].values))  |
             set(itertools.chain.from_iterable(df["базовые ноты"].values)) |
             set(itertools.chain.from_iterable(df["запахи"].values)) |
             set(itertools.chain.from_iterable(df["характеристики аромата"].values)))
itemSmells.sort()
itemSmells.remove('nan')
for item in itemSmells:
    df[item] = df["верхние ноты"].map(lambda elem: 1 if item in elem else 0) | \
               df["ноты сердца"].map(lambda elem: 1 if item in elem else 0) | \
               df["базовые ноты"].map(lambda elem: 1 if item in elem else 0) | \
               df["запахи"].map(lambda elem: 1 if item in elem else 0) | \
               df["характеристики аромата"].map(lambda elem: 1 if item in elem else 0)

itemFamily = list(set(itertools.chain.from_iterable(df["семейства"].values)))
for item in itemFamily:
    df[item + " семейства"] = df["семейства"].map(lambda elem: 1 if item in elem else 0)

for smell in smellLike.keys().drop('семейство'):
    df[smell] = df["семейства"].map(lambda elem:
        1 if sum(row[smell] for row in smellLike.iloc if row['семейство'] in elem) else 0)

In [456]:
del df["верхние ноты"]
del df["ноты сердца"]
del df["базовые ноты"]
del df["запахи"]
del df["семейства"]
del df["время года"]
del df["характеристики аромата"]
del df["мл"]
del df["цена"]

In [457]:
display(df)

,название,пол,бренд,страна,сегмент,концентрация,стойкость минимум (ч),стойкость максимум (ч),шлейф,возраст (до 18),возраст (до 26),возраст (до 35),возраст (до 45),возраст (после 45),цена за 1 мл,лето,весна,зима,осень,альдегиды,амбра,амброксан,ананас,апельсин,апельсиновый цвет,арбуз,белые цветы,белый перец,бензоин,бергамот,береза,бобы,бумага,ваниль,ветивер,вишня,гардения,гвоздика,герань,грейпфрут,груша,древесный аккорд,дубовый мох,дыня,жасмин,женственный,жженый сахар,замша,зеленое яблоко,зеленый аккорд,злаки,иланг-иланг,имбирь,ирис,какао,карамель,кардамон,кашмеран,кедр,киви,клубника,кожа,кожаный аккорд,корень фиалки,кориандр,корица,коричневый сахар,кофе,красное яблоко,лаванда,ладан,ландыш,лен,лилия,лимон,личи,магнолия,малина,мандарин,миндаль,можжевельник,мох,мускатный орех,мускус,нарцисс,нежный,нероли,опопанакс,орхидея,османтус,палисандр,пачули,пион,повседневный,полынь,праздничный,ревень,роза,розмарин,розовый перец,сандал,сахар,свежий,сексуальный,силколид,сирень,сладкая вата,сладкий,слива,специи,спортивный,стойкий,тиаре,тимьян,тмин,тропические фрукты,тубероза,уд,фиалка,франжипани,цвет персика,цибетин,цикламен,цитрусы,чай,черная смородина,черника,черный перец,чистый,шалфей,шафран,элеми,яблоко,янтарь,ароматические семейства,пряные семейства,фужерные семейства,цитрусовые семейства,амбровые семейства,водные семейства,восточные семейства,ванильные семейства,сладкие семейства,цветочные семейства,древесные семейства,мускусные семейства,шипровые семейства,фруктовые семейства,активный,белый чай,благовоние,бодрящий,виноград,горный воздух,древесный,дым,жизнерадостный,зеленый чай,земля,игривый,интенсивный,куркума,лайм,легкий,листья,массивный,металл,морозная свежесть,морской,мягкий,насыщенный,ненавязчивый,необычный,огурец,орех,острый,папоротник,перец,персик,плотный,притягательный,пряно-горький,пряный,роскошный,сливочный,смола,согревающий,спокойный,сухая трава,сухой,табак,таинственный,телесный,теплый,терпкий,томный,тонизирующий,тонкий,торжественный,трава,тяжелый,черный чай,чувственный,шоколад,энергичный,яркий
0,Shalimar,1.0,Guerlain,Франция,0.5,0.8,0.166667,0.005,0.50,1,1,1,1,1,0.030009,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,1,0,1,0,0,1,1,0,1,1,0,0,1,1,0,0,0,0,1,0,0,1,0,1,1,1,1,0,0
1,3 L'Imperatrice,1.0,Dolce&Gabbana,Италия,0.5,0.4,1.000000,1.000,1.00,1,1,1,1,1,0.016605,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1001 Nights,1.0,Ajmal,ОАЭ,0.5,1.0,0.416667,0.012,0.50,1,1,1,1,1,0.001804,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,1,1,0,0,1,0,1,0,0,0,0,1,0,0,0,1,0,1,0,1,1,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0
3,Sabina,1.0,Arabesque Perfumes,ОАЭ,1.0,0.8,0.416667,0.012,0.50,0,0,1,1,1,0.068557,1,1,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Resolute Gold,0.0,Khalis,ОАЭ,0.0,0.6,NaN,NaN,0.50,0,0,0,1,1,0.000000,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,

In [458]:
nameArr = df["название"]
del df["название"]

#### Матрица смежности для семейств (1 уровень)

In [459]:
layer1 = {"восточные": 0, "древесные": 1, "цветочные": 2, "цитрусовые": 3}

treeLayer1 = np.zeros((len(layer1), len(layer1)))
treeLayer1[layer1["восточные"]][layer1["древесные"]] = treeLayer1[layer1["древесные"]][layer1["восточные"]] = 0.5
treeLayer1[layer1["восточные"]][layer1["цветочные"]] = treeLayer1[layer1["цветочные"]][layer1["восточные"]] = 0.7
treeLayer1[layer1["восточные"]][layer1["цитрусовые"]] = treeLayer1[layer1["цитрусовые"]][layer1["восточные"]] = 0.9
treeLayer1[layer1["древесные"]][layer1["цветочные"]] = treeLayer1[layer1["цветочные"]][layer1["древесные"]] = 0.7
treeLayer1[layer1["древесные"]][layer1["цитрусовые"]] = treeLayer1[layer1["цитрусовые"]][layer1["древесные"]] = 0.9
treeLayer1[layer1["цветочные"]][layer1["цитрусовые"]] = treeLayer1[layer1["цитрусовые"]][layer1["цветочные"]] = 0.7

#### Матрица смежности для семейств (2 уровень)

In [460]:
layer2 = {"цветочные": 0, "древесные": 1, "пряные": 2, "ванильные": 3, "фужерные": 4, "водные": 5, "свежие": 6, "фруктовые": 7, "сладкие": 8, "ароматические": 9}

treeLayer2 = np.zeros((len(layer2), len(layer2)))
treeLayer2[layer2["цветочные"]][layer2["древесные"]] = treeLayer2[layer2["древесные"]][layer2["цветочные"]] = 0.5
treeLayer2[layer2["цветочные"]][layer2["пряные"]] = treeLayer2[layer2["пряные"]][layer2["цветочные"]] = 0.5
treeLayer2[layer2["цветочные"]][layer2["ванильные"]] = treeLayer2[layer2["ванильные"]][layer2["цветочные"]] = 0.3
treeLayer2[layer2["цветочные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["цветочные"]] = 0.6
treeLayer2[layer2["цветочные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["цветочные"]] = 0.5
treeLayer2[layer2["цветочные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["цветочные"]] = 0.3
treeLayer2[layer2["цветочные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["цветочные"]] = 0.3
treeLayer2[layer2["цветочные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["цветочные"]] = 0.2
treeLayer2[layer2["цветочные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["цветочные"]] = 0.3

treeLayer2[layer2["древесные"]][layer2["пряные"]] = treeLayer2[layer2["пряные"]][layer2["древесные"]] = 0.4
treeLayer2[layer2["древесные"]][layer2["ванильные"]] = treeLayer2[layer2["ванильные"]][layer2["древесные"]] = 0.5
treeLayer2[layer2["древесные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["древесные"]] = 0.3
treeLayer2[layer2["древесные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["древесные"]] = 0.6
treeLayer2[layer2["древесные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["древесные"]] = 0.5
treeLayer2[layer2["древесные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["древесные"]] = 0.6
treeLayer2[layer2["древесные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["древесные"]] = 0.4
treeLayer2[layer2["древесные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["древесные"]] = 0.4

treeLayer2[layer2["пряные"]][layer2["ванильные"]] = treeLayer2[layer2["ванильные"]][layer2["пряные"]] = 0.5
treeLayer2[layer2["пряные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["пряные"]] = 0.3
treeLayer2[layer2["пряные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["пряные"]] = 0.6
treeLayer2[layer2["пряные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["пряные"]] = 0.6
treeLayer2[layer2["пряные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["пряные"]] = 0.6
treeLayer2[layer2["пряные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["пряные"]] = 0.3
treeLayer2[layer2["пряные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["пряные"]] = 0.3

treeLayer2[layer2["ванильные"]][layer2["фужерные"]] = treeLayer2[layer2["фужерные"]][layer2["ванильные"]] = 0.5
treeLayer2[layer2["ванильные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["ванильные"]] = 0.7
treeLayer2[layer2["ванильные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["ванильные"]] = 0.6
treeLayer2[layer2["ванильные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["ванильные"]] = 0.4
treeLayer2[layer2["ванильные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["ванильные"]] = 0.3
treeLayer2[layer2["ванильные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["ванильные"]] = 0.5

treeLayer2[layer2["фужерные"]][layer2["водные"]] = treeLayer2[layer2["водные"]][layer2["фужерные"]] = 0.7
treeLayer2[layer2["фужерные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["фужерные"]] = 0.7
treeLayer2[layer2["фужерные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["фужерные"]] = 0.7
treeLayer2[layer2["фужерные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["фужерные"]] = 0.5
treeLayer2[layer2["фужерные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["фужерные"]] = 0.5

treeLayer2[layer2["водные"]][layer2["свежие"]] = treeLayer2[layer2["свежие"]][layer2["водные"]] = 0.3
treeLayer2[layer2["водные"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["водные"]] = 0.7
treeLayer2[layer2["водные"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["водные"]] = 0.6
treeLayer2[layer2["водные"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["водные"]] = 0.6

treeLayer2[layer2["свежие"]][layer2["фруктовые"]] = treeLayer2[layer2["фруктовые"]][layer2["свежие"]] = 0.5
treeLayer2[layer2["свежие"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["свежие"]] = 0.6
treeLayer2[layer2["свежие"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["свежие"]] = 0.3

treeLayer2[layer2["фруктовые"]][layer2["сладкие"]] = treeLayer2[layer2["сладкие"]][layer2["фруктовые"]] = 0.3
treeLayer2[layer2["фруктовые"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["фруктовые"]] = 0.3

treeLayer2[layer2["сладкие"]][layer2["ароматические"]] = treeLayer2[layer2["ароматические"]][layer2["сладкие"]] = 0.6

#### Матрица смежности для семейств (3 уровень)

In [461]:
layer3 = {"свежие": 0, "сладкие": 1, "мускусные": 2, "шипровые": 3, "амбровые": 4, "фруктовые": 5, "-": 6}

treeLayer3 = np.zeros((len(layer3), len(layer3)))
treeLayer3[layer3["свежие"]][layer3["сладкие"]] = treeLayer3[layer3["сладкие"]][layer3["свежие"]] = 0.6
treeLayer3[layer3["свежие"]][layer3["мускусные"]] = treeLayer3[layer3["мускусные"]][layer3["свежие"]] = 0.7
treeLayer3[layer3["свежие"]][layer3["шипровые"]] = treeLayer3[layer3["шипровые"]][layer3["свежие"]] = 0.7
treeLayer3[layer3["свежие"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["свежие"]] = 0.7
treeLayer3[layer3["свежие"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["свежие"]] = 0.6
treeLayer3[layer3["свежие"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["свежие"]] = 1

treeLayer3[layer3["сладкие"]][layer3["мускусные"]] = treeLayer3[layer3["мускусные"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["шипровые"]] = treeLayer3[layer3["шипровые"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["сладкие"]] = 0.4
treeLayer3[layer3["сладкие"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["сладкие"]] = 1

treeLayer3[layer3["мускусные"]][layer3["шипровые"]] = treeLayer3[layer3["шипровые"]][layer3["мускусные"]] = 0.2
treeLayer3[layer3["мускусные"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["мускусные"]] = 0.6
treeLayer3[layer3["мускусные"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["мускусные"]] = 0.6
treeLayer3[layer3["мускусные"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["мускусные"]] = 1

treeLayer3[layer3["шипровые"]][layer3["амбровые"]] = treeLayer3[layer3["амбровые"]][layer3["шипровые"]] = 0.6
treeLayer3[layer3["шипровые"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["шипровые"]] = 0.6
treeLayer3[layer3["шипровые"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["шипровые"]] = 1

treeLayer3[layer3["амбровые"]][layer3["фруктовые"]] = treeLayer3[layer3["фруктовые"]][layer3["амбровые"]] = 0.7
treeLayer3[layer3["амбровые"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["амбровые"]] = 1

treeLayer3[layer3["фруктовые"]][layer3["-"]] = treeLayer3[layer3["-"]][layer3["фруктовые"]] = 1

In [462]:
layer = [layer1, layer2, layer3]
tree = [treeLayer1, treeLayer2, treeLayer3]

#### Получение расстояний

In [483]:
def _complete(v1, v2):
    for i in range(len(v1) - len(v2)):
        v2.append("-")
    return v2

def complete(v1, v2):
    n1, n2 = len(v1), len(v2)
    if n1 > n2:
        v2 = _complete(v1, v2)
    elif n1 < n2:
        v1 = _complete(v2, v1)
    return v1, v2

# Расстояния
def getDistance(v1, v2, nPow):
    res = 0
    for i in range(len(v1)):
        if isnan(v1[i]) or isnan(v2[i]):
            continue
        res += pow(abs(v1[i] - v2[i]), nPow)
    return pow(res, 1 / nPow)

# Манхэттенское расстояние
def getManhattanDistance(v1, v2):
    return getDistance(v1, v2, 1)

# Евклидово расстояние
def getEuclideanDistance(v1, v2):
    return getDistance(v1, v2, 2)

# Косинусное 
def getCos(v1, v2):
    v1T, v2T = v1.copy(), v2.copy()
    n = len(v1)
    indArr = [i for i, (elem1, elem2) in enumerate(zip(v1T, v2T)) if isnan(elem1) or isnan(elem2)]
    v1T[:] = [elem for i, elem in enumerate(v1T) if i not in indArr]
    v2T[:] = [elem for i, elem in enumerate(v2T) if i not in indArr]
    return dot(v1T, v2T) / (norm(v1T) * norm(v2T))

# Расстояние по дереву
def getTreeDistance(v1, v2):
    res = 0
    resArr = []
    v1T, v2T = v1, v2
    if len(v1) != len(v2):
        v1T, v2T = complete(v1, v2)
    for i in range(len(v1T)):
        res += tree[i][layer[i][v1T[i]]][layer[i][v2T[i]]]
    
    return res / len(tree)

# Сравнение брендов
def getBrandDistance(v1, v2):
    return 0 if v1[0] == v2[0] else 1

# Сравнение стран
def getCountryDistance(v1, v2):
    return countryMatr[countryDict[v1]][countryDict[v2]]

def getDifference(id, matr, nameArr):
    data = matr[id]
    res = pd.DataFrame(zip(data, nameArr), index=np.arange(len(matr)), columns=["Расстояние", "Название"])
    return res.sort_values("Расстояние")

In [501]:
from turtle import color
from unicodedata import name
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
import plotly.express as px


def calcDistance(f, df):
    matrData = df.values.tolist()
    n = len(matrData)
    matrRes = np.zeros((n, n))
    for i in range(n):
        for j in range(i, n):
            matrRes[i][j] = matrRes[j][i] = f(matrData[i], matrData[j])
    return matrRes / matrRes.max()   


def calcDistanceCompined(df, dfTree, dfMan, dfStatParams):
    matrTree = calcDistance(getTreeDistance, dfTree)
    # matrCos = 1 - calcDistance(getCos, dfMan)
    matrEucl = calcDistance(getEuclideanDistance, dfStatParams)
    matrMan = calcDistance(getManhattanDistance, dfMan)
    matrBrand = calcDistance(getBrandDistance, df["бренд"])
    matrCountry = calcDistance(getCountryDistance, df["страна"])

    xTree = matrTree.max()
    # xCos = matrCos.max()
    xEuci = matrEucl.max()
    xMan = matrMan.max()
    xBrand = matrBrand.max()
    xCountry = matrCountry.max()


    kTree, kEuci, kBrand, kCountry = 0.5, 10, 2, 2

    return (matrMan + kTree * matrTree + matrEucl + kBrand * matrBrand + kCountry * matrCountry) / (xMan + kTree * xTree + xEuci + kBrand * xBrand + kCountry * xCountry)


def draw(matrRes, nameArr, title, color='Inferno'):
    fig = px.imshow(matrRes, x=nameArr, y=nameArr, color_continuous_scale=color, title=title)
    fig.update_layout(width=1000, height=1200)
    fig.update_traces(text=nameArr)
    fig.update_xaxes(side="top")
    fig.show()

### Демонстрация расстояний

#### Манхэттонское

In [465]:
dfMan = df.copy()
excludeFields = ["стойкость минимум (ч)", "стойкость максимум (ч)", "возраст (до 18)", "возраст (до 26)", "возраст (до 35)", 
                 "возраст (до 45)", "возраст (после 45)", "цена за 1 мл", "пол", "бренд", 
                 "страна", "сегмент", "концентрация"]
for elem in excludeFields:
    del dfMan[elem]


matrRes = calcDistance(getManhattanDistance, dfMan)
draw(matrRes, nameArr, "Манхэттенское расстояние")
# getDifference(11, matrRes, nameArr)

#### Евклидово расстояние

In [466]:
dfStatParams = pd.DataFrame(columns=excludeFields, data=df[excludeFields].values)
del dfStatParams["бренд"]
del dfStatParams["страна"]

matrRes = calcDistance(getEuclideanDistance, dfStatParams)
draw(matrRes, nameArr, "Евклидово расстояние")
# getDifference(11, matrRes, nameArr)

#### Расстояние по брендам

In [498]:
matrRes = calcDistance(getBrandDistance, df["бренд"])
draw(matrRes, nameArr, "Расстояние по брендам")

#### Расстояние по парфюмерным школам

In [496]:
matrRes = calcDistance(getCountryDistance, df["страна"])
draw(matrRes, nameArr, "Расстояние по парфюмерным школам")

#### Косинусное сходство

In [469]:
matrRes = calcDistance(getCos, dfMan)
draw(matrRes, nameArr, "Косинусное подобие", "Inferno_r")
# getDifference(11, matrRes, nameArr)

#### Расстояние по дереву

In [470]:
matrRes = calcDistance(getTreeDistance, dfTree["семейства"])
draw(matrRes, nameArr, "Расстояние по дереву")
# display(dfTree.sort_values("семейства"))


,название,семейства
18,Prada Day For Night,"[восточные, ванильные, амбровые]"
12,New Your Amber,"[восточные, ванильные, амбровые]"
13,Renata,"[восточные, ванильные, фруктовые]"
11,Samsara Eau de Parfum,"[восточные, древесные, -]"
0,Shalimar,"[восточные, пряные, -]"
20,The Only One,"[восточные, цветочные, -]"
10,Ange ou Demon,"[восточные, цветочные, -]"
19,Escentric Molecules Escentric 02,"[восточные, цветочные, -]"
8,Bad Boy,"[древесные, пряные, -]"
15,MOSCHINO Toy Boy,"[древесные, пряные, -]"


#### Кобинированная мера

In [502]:
draw(calcDistanceCompined(df, dfTree["семейства"], dfMan, dfStatParams), nameArr, "Кобинированная мера")